In [2]:
import os
import requests
import pandas as pd
from joblib import dump
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
DUNE_API_KEY = os.getenv("DUNE_API_KEY")

# Debug: Check if API key is loaded
if not DUNE_API_KEY:
    print("❌ API Key not found in environment variables")
    print("Please check your .env file contains: DUNE_API_KEY=your_key_here")
    exit()

# Check if this is a Dune Sim API key (won't work with main Dune API)
if DUNE_API_KEY.startswith('sim_'):
    print("❌ You're using a Dune Sim API key, but you need a regular Dune Analytics API key")
    print("Please get an API key from https://dune.com → Settings → API Keys")
    exit()

print(f"✅ API Key loaded: {DUNE_API_KEY[:8]}{'*' * (len(DUNE_API_KEY) - 8)}")


# QUERY_ID = "5591502" for top 100 whale actions
QUERY_ID = "5591694" # for 30 days whale count
BASE_URL = f"https://api.dune.com/api/v1/query/{QUERY_ID}/results"

# Set headers
headers = {
    "X-Dune-Api-Key": DUNE_API_KEY,
    "accept": "application/json"
}

# Fetch results
print("📡 Fetching whale transactions from Dune...")
try:
    res = requests.get(BASE_URL, headers=headers, timeout=30)
    
    if res.status_code == 401:
        print("❌ Authentication failed. Please check your API key.")
        print("Make sure you have a valid Dune Analytics API key (not Dune Sim)")
        print("Get one from: https://dune.com → Settings → API Keys")
        exit()
    elif res.status_code == 404:
        print(f"❌ Query ID {QUERY_ID} not found. Please verify the query ID.")
        exit()
    elif res.status_code != 200:
        print(f"❌ Error: {res.status_code}")
        print(f"Response: {res.text}")
        exit()
    
    print("✅ Successfully connected to Dune API")
    
except requests.exceptions.Timeout:
    print("❌ Request timed out. Please try again.")
    exit()
except requests.exceptions.RequestException as e:
    print(f"❌ Request failed: {e}")
    exit()

# Parse response
try:
    data = res.json()
    if 'result' not in data or 'rows' not in data['result']:
        print("❌ Unexpected response format from Dune API")
        print(f"Response keys: {list(data.keys())}")
        exit()
    
    records = data['result']['rows']
    
except ValueError as e:
    print(f"❌ Failed to parse JSON response: {e}")
    exit()

# Convert to DataFrame
df_whales = pd.DataFrame(records)

if df_whales.empty:
    print("⚠️ No whale transactions found.")
else:
    print(f"✅ Retrieved {len(df_whales)} transactions.")
    print(f"Columns: {list(df_whales.columns)}")

# Save to disk
save_dir = "C:/Users/DELL/Documents/Data Science and Analytics/Mini Capstone/minic/Database"
os.makedirs(save_dir, exist_ok=True)

try:
    dump(df_whales, os.path.join(save_dir, "whales_wif.joblib"))
    print(f"📦 Data saved to {os.path.join(save_dir, 'whales_wif.joblib')}")
except Exception as e:
    print(f"❌ Failed to save data: {e}")

✅ API Key loaded: jxqqfZYK************************
📡 Fetching whale transactions from Dune...
✅ Successfully connected to Dune API
✅ Retrieved 31 transactions.
Columns: ['day', 'whale_tx_count']
📦 Data saved to C:/Users/DELL/Documents/Data Science and Analytics/Mini Capstone/minic/Database\whales_wif.joblib
